<a href="https://colab.research.google.com/github/pashanushkarev/PIMonteCarlo/blob/main/PI_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycuda

     |████████████████████████████████| 1.6MB 15.5MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621320 sha256=5b30e9839ddc37949542d5556a470ddf0eb4064157d760dbcc438b798ffec916
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=58d3b6ae1dd1e3013aa21305fdb9e946f1fdd2a649cbb154bd6df63afa86f8d5
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [ ]:
import numpy as np
import math
import time
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule


In [ ]:
kernel = SourceModule("""
 __global__ void PiGPU(double *x, double *y, double *count) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 
        int threadCount = gridDim.x * blockDim.x;
        int n = 1000000;
        int result = 0;
        for (int i = idx; i < n; i += threadCount) {
                if (x[i] * x[i] + y[i] * y[i] < 1) {result++;}
        }
        atomicAdd(count , result);
}
""")

In [ ]:
def PiMonteCarloCPU(n):
    result = 0
    for i in range(n):
        if x[i] ** 2 + y[i] ** 2 < 1: result += 1  
    return result*4/N

In [ ]:
N = 1000000
x, y = np.random.random(N), np.random.random(N)
block_size = (256, 1, 1)
grid_size = (int(N / (128 * block_size[0])), 1)
result = np.zeros(1)
PiMonteCarloGPU = kernel.get_function("PiGPU")

In [ ]:
start_time_cpu = time.time()
PiMonteCarloCPU(N)
end_time_cpu = time.time() - start_time_cpu
print(f"Time cpu: {end_time_cpu} sec.")
print(f"Result cpu:", PiMonteCarloCPU(N))

Time cpu: 1.2076737880706787 sec.
Result cpu: 3.142092


In [ ]:
start_time_gpu = time.time()
PiMonteCarloGPU(drv.In(x), drv.In(y),drv.Out(result), block = block_size, grid = grid_size)
end_time_gpu = time.time() - start_time_gpu
print(f"Time gpu: {end_time_gpu} sec.")
print(f"Result gpu:", float(result*4/N))

Time gpu: 0.007369518280029297 sec.
Result gpu: 3.142092


In [ ]:
SpeedGPU2CPU = end_time_cpu/end_time_gpu
print(f"Ускорение GPU перед CPU: {SpeedGPU2CPU}")

Ускорение GPU перед CPU: 163.8741831122614
